<a href="https://colab.research.google.com/github/flowni/podcast-transcription/blob/main/simple_web_transcription_whisper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup Runtime

In [ ]:
# install requirements
!pip install gradio
!pip install openai-whisper

In [ ]:
# import packages
import gradio as gr
import whisper
import os
import requests
from urllib.parse import urlparse

# MP3 Download Function

In [9]:
def download_mp3(url, download_folder="downloads"):
  try:
      # Create the download folder if it doesn't exist
      if not os.path.exists(download_folder):
          os.makedirs(download_folder)

      response = requests.get(url, stream=True)
      response.raise_for_status()

      # Extract filename from the URL
      parsed_url = urlparse(url)
      filename = os.path.basename(parsed_url.path)

      # Save the file to the download folder with the extracted filename
      file_path = os.path.join(download_folder, filename)

      with open(file_path, 'wb') as file:
          for chunk in response.iter_content(chunk_size=8192):
              file.write(chunk)

      print(f"Downloaded: {file_path}")
      return file_path

  except requests.exceptions.RequestException as e:
      print(f"Error downloading: {e}")
      return None

# Load Model and Transcription Functions

In [ ]:
# load whisper model (automatically in GPU if available)
model = whisper.load_model("medium") # see available models here: https://github.com/openai/whisper#available-models-and-languages

In [6]:
def transcribe(audio):
  # calls transcribe function of whisper
  use_fp16 = model.device.type == "cpu" # CPU cannot handle fp16
  try:
      transcription = model.transcribe(audio, fp16=use_fp16)
  except Exception as e:
      print("error", str(e))
  return transcription


def transcribe_url(url):
  # calls downloader and transcribe function
  transcription = {"text": "Error in process"} # only shows on error
  try:
      mp3_file_path = download_mp3(url)
      transcription = transcribe(mp3_file_path)
  except Exception as e:
      print("error", str(e))
  return transcription["text"]

# Launch Gradio App
Run this cell.<br/>
Then use the app in this Notebook or with the public link provided in the output line like:<br/> `Running on public URL: https://12345678912345678.gradio.live`

In [ ]:
# Run app
inputs = gr.components.Textbox(lines=1, label="Audio URL")
outputs = gr.components.Textbox(label="Transcription")

demo = gr.Interface(
    fn=transcribe_url,
    inputs=inputs,
    outputs=outputs,
    allow_flagging="never"
)

demo.queue().launch(share=True, debug=True)